In [1]:
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, DayLocator
import seaborn as sns

In [2]:
import nltk

nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raisul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raisul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Read the tweets file

In [3]:
tweets_df= pd.read_csv("english_tweets.csv")
tweets_df.head()

,Unnamed: 0,created_at,tweet,likes,retweet_count,country,continent,state,candidate,cleaned_tweet,detected_language
0,2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,US,North America,Oregon,trump_df,trump as a student i used to hear for years fo...,en
1,4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,US,North America,District of Columbia,trump_df,you get a tie and you get a tie trump s rally ...,en
2,5,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,US,North America,California,trump_df,her 15 minutes were over long time ago omarosa...,en
3,7,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,US,North America,Ohio,trump_df,there wont be many of them unless you all have...,en
4,8,2020-10-15 00:00:20,One of the single most effective remedies to e...,0.0,0.0,US,North America,Pennsylvania,trump_df,one of the single most effective remedies to e...,en


In [4]:
stp_words= stopwords.words('english')
print(stp_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

First create a function to remove the stop words from the cleaned tweets

In [5]:
def TweetFurtherCleaning(tweet):
    cleanTweet= ' '.join(word for word in tweet.split() if word not in stp_words)
    return cleanTweet

In [6]:
tweets_df["cleaned_tweet"]= tweets_df["cleaned_tweet"].apply(TweetFurtherCleaning)

In [7]:
tweets_df.head()

,Unnamed: 0,created_at,tweet,likes,retweet_count,country,continent,state,candidate,cleaned_tweet,detected_language
0,2,2020-10-15 00:00:02,"#Trump: As a student I used to hear for years,...",2.0,1.0,US,North America,Oregon,trump_df,trump student used hear years ten years heard ...,en
1,4,2020-10-15 00:00:08,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,US,North America,District of Columbia,trump_df,get tie get tie trump rally iowa,en
2,5,2020-10-15 00:00:17,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,US,North America,California,trump_df,15 minutes long time ago omarosa never represe...,en
3,7,2020-10-15 00:00:18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,US,North America,Ohio,trump_df,wont many unless voting god prevails bo corrup...,en
4,8,2020-10-15 00:00:20,One of the single most effective remedies to e...,0.0,0.0,US,North America,Pennsylvania,trump_df,one single effective remedies eradicate anothe...,en


In [8]:
tweets_df.shape

(293929, 11)

Emotion analysis

In [10]:
from pysentimiento import create_analyzer
import transformers

transformers.logging.set_verbosity(transformers.logging.ERROR)

analyzer = create_analyzer(task="emotion", lang="en")

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

c:\Users\Raisul\Anaconda3\envs\my_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Raisul\.cache\huggingface\hub\models--finiteautomata--bertweet-base-emotion-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [11]:
def sentiment(input_text):
    sentiment_result= analyzer.predict(input_text)
    return sentiment_result

In [12]:
result= tweets_df["cleaned_tweet"].apply(sentiment)

In [13]:
result

0         AnalyzerOutput(output=others, probas={others: ...
1         AnalyzerOutput(output=others, probas={others: ...
2         AnalyzerOutput(output=others, probas={others: ...
3         AnalyzerOutput(output=others, probas={others: ...
4         AnalyzerOutput(output=others, probas={others: ...
                                ...                        
293924    AnalyzerOutput(output=others, probas={others: ...
293925    AnalyzerOutput(output=disgust, probas={disgust...
293926    AnalyzerOutput(output=joy, probas={joy: 0.795,...
293927    AnalyzerOutput(output=disgust, probas={disgust...
293928    AnalyzerOutput(output=others, probas={others: ...
Name: cleaned_tweet, Length: 293929, dtype: object

In [14]:
result_df= pd.DataFrame(result)

In [15]:
result_df.rename(columns={"cleaned_tweet": "sentiment"}, inplace=True)

In [16]:
result_df

,sentiment
0,"AnalyzerOutput(output=others, probas={others: ..."
1,"AnalyzerOutput(output=others, probas={others: ..."
2,"AnalyzerOutput(output=others, probas={others: ..."
3,"AnalyzerOutput(output=others, probas={others: ..."
4,"AnalyzerOutput(output=others, probas={others: ..."
...,...
293924,"AnalyzerOutput(output=others, probas={others: ..."
293925,"AnalyzerOutput(output=disgust, probas={disgust..."
293926,"AnalyzerOutput(output=joy, probas={joy: 0.795,..."
293927,"AnalyzerOutput(output=disgust, probas={disgust..."


save as csv

In [17]:
result_df.to_csv("pysentimiento_emotion.csv")